In [20]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import shap
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time

def benchmark_dict_print(output_dict):
    for key in output_dict:
        #samples
        print(key+": ")
        for key_inner in output_dict[key]:
            #features
            print(key_inner+": ")
            
            #per informative
            time_str = ""
            
            i = 0
            for avg_time in output_dict[key][key_inner]["Average time"]:
                
                i = i+1
                
                if i ==len(output_dict[key][key_inner]["Average time"]):
                    time_str = time_str + str(avg_time)+"s | "
                else:
                    time_str = time_str + str(avg_time)+"s - "
                  
            key_strs = ""
            for key_inner_inner in output_dict[key][key_inner]: 
                if key_inner_inner != 'Average time':
                    key_strs = key_strs + str(output_dict[key][key_inner][key_inner_inner]["found_informative_features"]) + " ("+str(output_dict[key][key_inner][key_inner_inner]["outputted_noise_features"])+") / "+str(output_dict[key][key_inner][key_inner_inner]["informative_features"]) + " | "
                
            print(time_str+key_strs)
                 

def output_dict_to_df(output_dict):
    df = pd.DataFrame(columns=["n_samples","total_features","informative_features","time","seed","found_informative_features","outputted_noise_features"])
    for samples in output_dict.keys():
        for features in output_dict[samples].keys():
            i = 0
            for informative in output_dict[samples][features].keys():
                if informative != 'Average time':
                    temp = pd.DataFrame.from_dict(output_dict[samples][features][informative])
                    temp["n_samples"]=int(samples)
                    temp["total_features"]=int(features)
                    temp = temp.reset_index()
                    temp = temp.rename(columns={"index":"seed"})
                    temp["time"]=output_dict[samples][features]['Average time'][i]
                    i = i+1
                    df = df.append(temp) 

    return df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Simple testing

## Simulation dataset creation

In [54]:
n_features = 250 #20,50,100,250,500
n_informative = int(0.50*n_features) #5%,10%,33%,50%,90%
n_samples = int(5000/(1-0.33))+1 #7463#5000

X, y = make_classification(n_samples=n_samples, n_classes=3, n_features=n_features, n_informative=n_informative, n_redundant=0, n_repeated = 0,shuffle=False)
#X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative,random_state=4,shuffle=False)
X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=42)
X_train

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_240,col_241,col_242,col_243,col_244,col_245,col_246,col_247,col_248,col_249
4529,5.536752,3.329688,-0.374753,-7.275569,-0.624893,-1.499693,1.977829,-3.071376,3.080245,2.327211,...,1.327366,1.570263,0.475864,0.439342,2.239473,0.075484,-0.964398,0.912074,1.726683,-0.091777
2897,-10.697341,8.409849,-0.952889,-0.095059,-1.480137,0.324743,-3.847186,7.785810,-0.887215,-8.101364,...,0.325669,-0.114481,-0.672617,0.474071,0.123575,-1.914536,0.454361,-0.891399,1.072733,1.710560
6361,13.538943,-4.024491,0.750913,5.088994,13.657020,-3.768372,-9.089884,-8.160700,-6.104018,-3.869485,...,0.033836,0.970215,2.080081,-0.949400,-1.449323,-1.315960,0.902113,-0.712971,-0.050301,0.695432
1654,2.615468,-7.541920,-4.764081,3.045856,1.201153,-8.023545,4.757222,-5.144326,3.494003,-3.213149,...,-1.194840,-0.906861,0.493831,0.204400,-0.123042,-0.169966,0.416155,1.816210,1.427515,0.361296
6111,-1.437300,0.841735,2.732837,-6.163023,-0.197919,-2.828225,-2.501394,0.964650,-0.818581,6.158021,...,1.899658,0.001285,0.134793,0.553387,0.796371,-0.513223,1.142012,0.662710,0.405450,0.391325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,-10.431838,-2.359357,5.385657,11.688228,3.107147,-0.554976,-2.232681,15.605095,4.643017,-2.182702,...,-1.016582,0.342735,-1.268437,-2.181032,-0.437150,-1.565214,-2.694955,-0.193151,1.229937,-0.358881
5226,1.935839,-13.056533,4.457105,0.561530,8.172381,-4.321729,7.176320,4.755838,-0.864749,0.589921,...,-0.537557,0.604720,0.244738,-0.739641,-0.587323,-1.992174,1.118194,-1.687780,-0.601554,-0.147619
5390,3.296154,7.531825,4.672194,-2.012609,-7.853813,-9.631838,3.215517,4.077291,5.514906,-11.953926,...,0.833640,-0.645731,0.144210,-1.317162,0.134549,1.392515,-0.174187,1.394752,0.219805,-0.845388
860,-6.230092,1.335155,-5.594419,3.818041,-1.689493,1.495002,11.434524,-2.149342,-0.221712,-9.969088,...,0.049193,0.048176,-1.320313,0.703710,0.121755,1.012424,0.218005,1.129117,-0.876136,-1.608275


## Single powershap test

In [55]:
import sys
sys.path.append("../powershap")

from powershap import PowerShap


from catboost import CatBoostClassifier,CatBoostRegressor
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV,LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier

selector = PowerShap(
    model = CatBoostClassifier(verbose=0, n_estimators=250,use_best_model=True),#LogisticRegressionCV(),#GradientBoostingClassifier(),#CatBoostClassifier(verbose=0, n_estimators=250),
    #model = CatBoostRegressor(verbose=0, n_estimators=0,use_best_model=True),
    verbose=True,
)
selector.fit(X_train, y_train)

Starting powershap
Automatic mode enabled: Finding the minimal required powershap iterations for significance of 0.01.


A Jupyter Widget


10 iterations were already sufficient as only 10 iterations were required for the current  power_alpha = 0.01.
Done!


PowerSHAP(model=<catboost.core.CatBoostClassifier object at 0x00000144F02BB1C0>,
          verbose=True)

In [58]:
selector._processed_shaps_df.reset_index()[50:100]

,index,impact,p_value,effect_size,power_0.01_alpha,0.99_power_its_req
50,col_78,0.065015,0.0,-10.262464,1.0,2.847519
51,col_54,0.064240,0.0,-6.614742,1.0,3.354223
52,col_20,0.064163,0.0,-5.652398,1.0,3.608863
53,col_73,0.063961,0.0,-7.584173,1.0,3.169292
54,col_82,0.063334,0.0,-8.125184,1.0,3.086570
55,col_24,0.062137,0.0,-6.171394,1.0,3.460355
56,col_4,0.062086,0.0,-6.486940,1.0,3.383153
57,col_25,0.061776,0.0,-5.115298,1.0,3.800096
58,col_0,0.060734,0.0,-5.411950,1.0,3.688944
59,col_97,0.059677,0.0,-7.725241,1.0,3.146538


## Powershap sklearn pipeline test

In [ ]:
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier

pipe = Pipeline(
    [
        (
            "selector",
            PowerShap(
                CatBoostClassifier(n_estimators=250,verbose=False,use_best_model=True), automatic=True, limit_automatic=100,#power_alpha=0.001,power_req_iterations=0.999,
                #CatBoostRegressor(n_estimators=250,verbose=False), automatic=True, limit_automatic=100,
            ),
        ),
        ("catboost", KNeighborsClassifier()),#(n_estimators=250,verbose=False)),
        #("catboost", CatBoostRegressor(n_estimators=250,verbose=False)),
    ]
)

pipe.fit(X_train, y_train)


from sklearn.metrics import accuracy_score,r2_score


print("Baseline", accuracy_score(KNeighborsClassifier().fit(X_train, y_train).predict(X_test), y_test))
#print("Baseline", r2_score(LinearRegression.fit(X_train, y_train).predict(X_test), y_test))


print("PowerShap feature selection:", accuracy_score(pipe.predict(X_test), y_test))
#print("PowerShap feature selection:", r2_score(pipe.predict(X_test), y_test))



In [ ]:
print("Baseline", accuracy_score(CatBoostClassifier(verbose=False,n_estimators=250).fit(X_train, y_train).predict(X_test), y_test))

In [ ]:
processed_shaps_df = pipe[0]._processed_shaps_df

In [ ]:
len(processed_shaps_df[processed_shaps_df.p_value<0.01])

# benchmarking

## Estimators = 50 Classification

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
import pprint
#sys.path.append("../powershap")

from powershap import PowerShap
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
estimators = 50#250

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerShap(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerShap(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
output_dict_to_df(output_dict).to_csv("estimators_50_Classification_output_df.csv",index=False)

## Estimators = 250 Classification

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
estimators = 250
hypercube = False

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerShap(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerShap(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
        if hypercube:
            output_dict_to_df(output_dict).to_csv("estimators_250_Classification_output_df.csv",index=False)
        else:
            output_dict_to_df(output_dict).to_csv("estimators_250_Classification_output_df_polytone.csv",index=False)

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
estimators = 250
hypercube = False

output_dict = {}

for n_samples in [5000]:
    output_dict[str(n_samples)]={}
    for n_features in [50,100]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            for n_redundant in [10,25,33,50]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                
                n_redundant_samples = int((n_features-int(n_informative/100*n_features))*(n_redundant/100))
                
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
                print("Redundant features: "+str(n_redundant_samples)+" ("+str(n_redundant)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    print("Seed "+str(random_seed))

                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=int(n_informative/100*n_features), n_redundant=n_redundant_samples, n_repeated = 0,shuffle=False,random_state=random_seed)
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                    start_time = time.time()
                    if regression_bool:
                        selector = PowerShap(
                            model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                            automatic=True
                        )
                    else:
                        selector = PowerShap(
                            model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                            automatic=True
                        )
                    selector.fit(X, y)

                    times.append(time.time() - start_time)

                    processed_shaps_df = selector._processed_shaps_df
                    print(50*"-")

                    found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                    found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)

                found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
                found_redundant = [np.sum(np.isin(X.columns.values[int(n_informative/100*n_features):int(n_informative/100*n_features)+n_redundant_samples],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)+n_redundant_samples])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found redundant: "+str(found_redundant) + "/"+str(n_redundant_samples))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")

                average_times.append(np.round(times,2))

                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features

                print(100*"=")

        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
estimators = 250
hypercube = False

output_dict = {}

for n_samples in [5000]:
    output_dict[str(n_samples)]={}
    for n_features in [50]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            for n_redundant in [10,25,33,50]:
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
                
                n_redundant_samples = int((n_features-int(n_informative/100*n_features))*(n_redundant/100))
                
                print("Amount of samples = "+str(n_samples))
                print("Total used features = "+str(n_features))
                print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
                print("Redundant features: "+str(n_redundant_samples)+" ("+str(n_redundant)+"%)")
                print("")
                
                found_features = []
                found_idx_features = []
                times = []
                for random_seed in [0,1,2,3,4]:
                    print("Seed "+str(random_seed))

                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, hypercube=hypercube, n_informative=int(n_informative/100*n_features), n_redundant=n_redundant_samples, n_repeated = 0,shuffle=False,random_state=random_seed)
                    X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                    start_time = time.time()
                    # if classification is False it is a Regression problem
                    model = CatBoostClassifier(verbose=0, n_estimators=250)
                    selector = BorutaShap(model=model,importance_measure='shap',classification=True)
                    selector.fit(X, y,verbose=False)

                    times.append(time.time() - start_time)

                    subset = selector.Subset()
                    print(50*"-")

                    found_features.append(len(subset.columns))
                    found_idx_features.append(subset.columns)

                found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
                found_redundant = [np.sum(np.isin(X.columns.values[int(n_informative/100*n_features):int(n_informative/100*n_features)+n_redundant_samples],f_list)) for f_list in found_idx_features]
                found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)+n_redundant_samples])) for f_list in found_idx_features]
                print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
                print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found redundant: "+str(found_redundant) + "/"+str(n_redundant_samples))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
                print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
                print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")

                average_times.append(np.round(times,2))

                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
                output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features

                print(100*"=")

        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")

## Estimators = 500 Classification

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from catboost import CatBoostClassifier,CatBoostRegressor

regression_bool=False
estimators = 500

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerShap(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerShap(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
        output_dict_to_df(output_dict).to_csv("estimators_500_Classification_output_df.csv",index=False)

## Estimators = 50 Regression

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from catboost import CatBoostClassifier,CatBoostRegressor

regression_bool=True
estimators = 50

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerShap(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerShap(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
        
output_dict_to_df(output_dict).to_csv("estimators_50_Regression_output_df.csv",index=False)

## Estimators = 250 regression

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=True
estimators = 250

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerShap(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerShap(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
        
    
output_dict_to_df(output_dict).to_csv("estimators_250_Regression_output_df.csv",index=False)

## Estimators = 500 regression

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from catboost import CatBoostClassifier,CatBoostRegressor

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=True
estimators = 500

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                if regression_bool:
                    selector = PowerShap(
                        model = CatBoostRegressor(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                else:
                    selector = PowerShap(
                        model = CatBoostClassifier(verbose=0, n_estimators=estimators,use_best_model=True),
                        automatic=True
                    )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
        
    
output_dict_to_df(output_dict).to_csv("estimators_500_Regression_output_df.csv",index=False)

## Chi-squared

In [19]:
from sklearn.feature_selection import chi2,f_classif,f_regression

regression_bool=False
hypercube = True

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,hypercube=hypercube,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                
                X = X+np.abs(X.min())

                selected_features = list(X.columns.values[np.where(chi2(X,y)[1]<0.01)[0]])

                times.append(time.time() - start_time)

                print(50*"-")
                
                found_features.append(len(selected_features))
                found_idx_features.append(selected_features)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
        if hypercube:
            output_dict_to_df(output_dict).to_csv("chisquared_output_df.csv",index=False)
        else:
            output_dict_to_df(output_dict).to_csv("chi_squared_polytone_output_df.csv",index=False)

Amount of samples = 1000
Total used features = 20
Informative features: 2 (10%)

Seed 0
--------------------------------------------------
Seed 1
--------------------------------------------------
Seed 2
--------------------------------------------------
Seed 3
--------------------------------------------------
Seed 4
--------------------------------------------------
Average time: 0.0 seconds
Found features: [1, 1, 1, 1, 1]
Found 1.0 of 2 informative features
0.0 of 1.0 outputted powershap features are noise features
Amount of samples = 1000
Total used features = 20
Informative features: 6 (33%)

Seed 0
--------------------------------------------------
Seed 1
--------------------------------------------------
Seed 2
--------------------------------------------------
Seed 3
--------------------------------------------------
Seed 4
--------------------------------------------------
Average time: 0.0 seconds
Found features: [4, 3, 4, 6, 3]
Found 4.0 of 6 informative features
0.0 of 4.0 

## F test

In [18]:
from sklearn.feature_selection import f_classif,f_regression

regression_bool=False
hypercube = True

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,hypercube=hypercube,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()

                if regression_bool:
                    selected_features = list(X.columns.values[np.where(f_classif(X,y)[1]<0.01)[0]])
                else:
                    selected_features = list(X.columns.values[np.where(f_regression(X,y)[1]<0.01)[0]])

                times.append(time.time() - start_time)

                print(50*"-")
                
                found_features.append(len(selected_features))
                found_idx_features.append(selected_features)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
        if hypercube:
            output_dict_to_df(output_dict).to_csv("f_classif_output_df.csv",index=False)
        else:
            output_dict_to_df(output_dict).to_csv("f_classif_polytone_output_df.csv",index=False)

Amount of samples = 1000
Total used features = 20
Informative features: 2 (10%)

Seed 0
--------------------------------------------------
Seed 1
--------------------------------------------------
Seed 2
--------------------------------------------------
Seed 3
--------------------------------------------------
Seed 4
--------------------------------------------------
Average time: 0.0 seconds
Found features: [1, 1, 1, 1, 2]
Found 1.0 of 2 informative features
0.2 of 1.2 outputted powershap features are noise features
Amount of samples = 1000
Total used features = 20
Informative features: 6 (33%)

Seed 0
--------------------------------------------------
Seed 1
--------------------------------------------------
Seed 2
--------------------------------------------------
Seed 3
--------------------------------------------------
Seed 4
--------------------------------------------------
Average time: 0.0 seconds
Found features: [4, 3, 4, 6, 3]
Found 4.0 of 6 informative features
0.0 of 4.0 

## LogisticRegressionCV

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifierCV,LinearRegression

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False
hypercube = False

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,hypercube=hypercube,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()

                selector = PowerShap(
                    model = LogisticRegressionCV(max_iter=1000),
                    automatic=True
                )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
        if hypercube:
            output_dict_to_df(output_dict).to_csv("logisticregressioncv_output_df.csv",index=False)
        else:
            output_dict_to_df(output_dict).to_csv("logisticregressioncv_output_df_polytone.csv",index=False)

## RandomForest

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from sklearn.ensemble import RandomForestClassifier

regression_bool=False

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()

                selector = PowerShap(
                    model = RandomForestClassifier(),
                    automatic=True
                )
                selector.fit(X, y)
                
                times.append(time.time() - start_time)
                
                processed_shaps_df = selector._processed_shaps_df
                print(50*"-")
                
                found_features.append(len(processed_shaps_df[processed_shaps_df.p_value<0.01]))
                found_idx_features.append(processed_shaps_df[processed_shaps_df.p_value<0.01].index.values)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
        print(100*"=")
    
output_dict_to_df(output_dict).to_csv("randomforest_output_df.csv",index=False)

In [ ]:
#Stopped simulation at 5000 samples, 20000 samples takes on average 300s per model! so at least 3000s per seed. 
output_dict_to_df(output_dict).to_csv("randomforest_output_df.csv",index=False)

## BorutaShap

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
from BorutaShap import BorutaShap

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False

output_dict = {}

for n_samples in [1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])

                start_time = time.time()
                
                # if classification is False it is a Regression problem
                model = CatBoostClassifier(verbose=0, n_estimators=250)
                selector = BorutaShap(model=model,importance_measure='shap',classification=True)

                selector.fit(X=X, y=y, verbose=False)
                subset = selector.Subset()
                
                times.append(time.time() - start_time)
                print(50*"-")
                
                found_features.append(len(subset.columns))
                found_idx_features.append(subset.columns)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
            
        output_dict_to_df(output_dict).to_csv("250_est_Catboost_borutashap_output_df.csv",index=False)
        
        print(100*"=")

## Shapicant

In [ ]:
from sklearn.datasets import make_classification,make_regression
from sklearn.model_selection import train_test_split
import sys
import time
#sys.path.append("../powershap")

from powershap import PowerShap
import shapicant

#n_features = 20 #20,50,100,250,500
#n_informative = int(0.10*n_features) #10%,33%,50%,90%
#n_samples = 1000#5000

regression_bool=False

output_dict = {}

for n_samples in [5000]:#1000,5000,20000]:
    output_dict[str(n_samples)]={}
    for n_features in [20,100,250,500]:#[20,50,100,250,500]:
        output_dict[str(n_samples)][str(n_features)]={}
        
        average_times = []
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        for n_informative in [10,33,50,90]:#[int(0.10*n_features),int(0.33*n_features),int(0.50*n_features),int(0.90*n_features)]:
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]={}
            print("Amount of samples = "+str(n_samples))
            print("Total used features = "+str(n_features))
            print("Informative features: "+str(int(n_informative/100*n_features))+" ("+str(n_informative)+"%)")
            print("")
            
            found_features = []
            found_idx_features = []
            times = []
            for random_seed in [0,1,2,3,4]:
                print("Seed "+str(random_seed))
                
                if regression_bool:
                    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=int(n_informative/100*n_features),random_state=random_seed,shuffle=False)
                else:
                    X, y = make_classification(n_samples=n_samples, n_classes=2, n_features=n_features, n_informative=int(n_informative/100*n_features), n_redundant=0, n_repeated = 0,shuffle=False,random_state=random_seed)
                X = pd.DataFrame(data=X, columns=[f"col_{i}" for i in range(n_features)])
                X["class"]=y
                X = X.reset_index()

                
                explainer_type = shap.TreeExplainer
                # if classification is False it is a Regression problem
                model = CatBoostClassifier(verbose=0, n_estimators=250,use_best_model=False)
                selector = shapicant.PandasSelector(model, explainer_type, random_state=42)

                train_idx,val_idx = train_test_split(X["index"].values,test_size=0.2,random_state = 0)

                X_train = X[X["index"].isin(train_idx)].copy(deep=True)[list(X.columns.values[1:-1])]
                X_val = X[X["index"].isin(val_idx)].copy(deep=True)[list(X.columns.values[1:-1])]
                Y_train =  X[X["index"].isin(train_idx)]["class"]

                # Run the feature selection
                # If we provide a validation set, SHAP values are computed on it, otherwise they are computed on the training set
                # We can also provide additional parameters to the underlying estimator's fit method through estimator_params
                
                start_time = time.time()
                
                selector.fit(X_train, Y_train, X_validation=X_val)

                subset = selector.get_features()
                p_values = selector.p_values_

                np.array(subset)

                times.append(time.time() - start_time)
                print(50*"-")
                
                found_features.append(len(subset))
                found_idx_features.append(subset)
                
            found_informative_features = [np.sum(np.isin(X.columns.values[:int(n_informative/100*n_features)],f_list)) for f_list in found_idx_features]
            found_noise_features = [np.sum(1-np.isin(f_list,X.columns.values[:int(n_informative/100*n_features)])) for f_list in found_idx_features]
            print("Average time: "+str(np.round(np.mean(times),2))+" seconds")
            print("Found features: "+str(found_features))#len(processed_shaps_df[processed_shaps_df.p_value<0.01])))
            print("Found "+str(np.mean(found_informative_features))+" of "+str(int(n_informative/100*n_features))+" informative features")
            print(str(np.mean(found_noise_features))+" of "+str(np.mean(found_features))+" outputted powershap features are noise features")
            
            average_times.append(np.round(times,2))
            
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["informative_features"]=int(n_informative/100*n_features)
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["found_informative_features"]=found_informative_features
            output_dict[str(n_samples)][str(n_features)][str(n_informative)+"%"]["outputted_noise_features"]=found_noise_features
            
            print(100*"=")
            
        output_dict[str(n_samples)][str(n_features)]["Average time"]=average_times
        benchmark_dict_print(output_dict)
            
        output_dict_to_df(output_dict).to_csv("shapicant_output_df.csv",index=False)
        
        print(100*"=")